In [1]:
# import os
# import numpy as np
# import cv2
# import pandas as pd
# import matplotlib.pyplot as plt
# from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
# import seaborn as sb
# from keras.utils.np_utils import to_categorical
# from keras.callbacks import EarlyStopping
# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, GlobalAveragePooling2D,AveragePooling2D
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.model_selection import train_test_split

In [1]:
import tensorflow as tf
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.keras import models
from keras.applications.resnet import layers
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPooling2D

In [2]:
images = []
ages = []
    
def preprocessing(given_img):
    # given_img = cv2.cvtColor(given_img, cv2.COLOR_BGR2GRAY)  # converting image to grayscale.
    # given_img = cv2.equalizeHist(given_img)
    given_img = given_img / 255  # normalizing image.
    given_img = cv2.resize(given_img, (64, 64))  # resizing it.
    return given_img

## Loading Data

In [3]:
# Please provide your own image path
path = "C:/Users/mrsal/Github Repositories/Dataset/Age Recognition/UTKFace"

for image in os.listdir(path):
    img = cv2.imread(path + "/" + image)
    images.append(img)
    ages.append(float(image.split("_")[0]))

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(images, ages, test_size=0.33, random_state=42)
y_train = np.array(y_train)
y_valid = np.array(y_valid)

X_train = np.array([preprocessing(x) for x in X_train]).reshape(len(X_train), 64, 64, 3)
X_valid = np.array([preprocessing(x) for x in X_valid]).reshape(len(X_valid), 64, 64, 3)

In [5]:
X_train.shape

(15884, 64, 64, 3)

## Creating and fitting the model

In [8]:
def create_model(input_shape):
    base_model = tf.keras.applications.ResNet50(
        include_top=False, weights='imagenet', input_shape=input_shape
    )

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='linear')(x)
    model = models.Model(inputs=base_model.input, outputs=predictions)
    return model

In [9]:
# Create the model and compile it
input_shape = (64, 64, 3)
model = create_model(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['mean_squared_error'])

In [10]:
# Train the model
stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_valid, y_valid), callbacks=[stop])
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_valid, y_valid))

Epoch 1/50
497/497 [==============================] - 23s 28ms/step - loss: 365.9109 - mean_squared_error: 365.9109 - val_loss: 296.9227 - val_mean_squared_error: 296.9227
Epoch 2/50
497/497 [==============================] - 12s 24ms/step - loss: 284.0000 - mean_squared_error: 284.0000 - val_loss: 280.3196 - val_mean_squared_error: 280.3196
Epoch 3/50
497/497 [==============================] - 12s 24ms/step - loss: 267.9428 - mean_squared_error: 267.9428 - val_loss: 290.5996 - val_mean_squared_error: 290.5996
Epoch 4/50
497/497 [==============================] - 12s 24ms/step - loss: 254.2588 - mean_squared_error: 254.2588 - val_loss: 263.3978 - val_mean_squared_error: 263.3978
Epoch 5/50
497/497 [==============================] - 12s 24ms/step - loss: 239.8108 - mean_squared_error: 239.8108 - val_loss: 267.3148 - val_mean_squared_error: 267.3148
Epoch 6/50
497/497 [==============================] - 12s 25ms/step - loss: 236.8174 - mean_squared_error: 236.8174 - val_loss: 229.3600 - v

In [12]:
import matplotlib.pyplot as plt
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_valid, y_valid)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

# Plot the training and validation loss/accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [10]:
# This code is in order to load the model. Make sure location and naming of a model is same, otherwise, please change below
model = tf.keras.models.load_model("./CNN_MODEL_64.h5")